# Animating a 3-D Robot: Inverse Kinematics and Cost-Function Minimization



In [1]:
# multimodal function
import numpy as np
import matplotlib.pyplot as plt

---
This notebook implements version of the method described in the following paper: 
- David E. Breen. Cost minimization for animated geometric models in computer graphics. The Journal of Visualization and Computer Animation, 8(4):201–220, 1997.
---

![](figs/robot_path.png)
**Figure 1:** Goal-oriented animation: The robot arm moves from one target location to another while avoiding obstacles. The yelllow spheres represent the targets. 

## The cost function 

The animation is achieved by using the gradient-descent algorithm to solve the following minimization problem: 

$$
\begin{align}
	\hat{\bf \Phi} = \arg_{{{\bf \Phi}}}\min  C\left({\bf \Phi}\right),
	\label{simple_motion_min}
\end{align}
$$

where the cost function $C$ is defined by:

$$
\begin{align}
	C\left({\bf \Phi}\right) = \underbrace{\|{\bf e}\left({\bf \Phi}\right) - {\bf g}\|}_{\text{goal attraction}} + \underbrace{\sum_{i=1}^{n} \mathcal{F}_R\left(\|{\bf e}\left({\bf \Phi}\right) - {\bf o}_i\|\right)}_{\text{obstacle-avoidance penalty}} + \underbrace{\sum_{j=1}^{m} \mathcal{L}\left(\phi_j\right)}_{\text{Joint-range limit}}.
	\label{simple_motion}
\end{align}
$$

Here, ${\bf g} = \left(x_g, y_g, z_g\right)^\mathsf{T}$ is the *goal location*, ${\bf o}_i$ is the location of obstacle $i$. The summation limits are $n$, which is the numeber obstacles, and $m$, which is the number of joints to be constrained. Function ${\bf e}\left({\bf \Phi}\right)$ computes the arm's *forward kinematics* and returns the location of the arm's tip ${\bf e} = \left( e_x, e_y, e_z \right)^\mathsf{T}$, i.e., the *end-effector's location,* given the arm's joint angles, ${\bf \Phi} = \left( \phi_1, \phi_2,  \phi_3, \phi_4\right)^\mathsf{T}$.   Function $\mathcal{F}_R$ is a collision-avoidance penalty field. It penalizes poses that take the end effector too close to an obstacle, i.e., beyond a pre-defined distance $R$. The third component of Equation $\ref{simple_motion}$  limits the range of each joint angle. Function $\mathcal{L}$ is another penalty function. Its value increases as the joint angle $\phi_j$ approaches its maximum or minimum limit. Outside these limits, $\mathcal{L}$ vanishes. 

Next, we describe the components of the cost function in  ($\ref{simple_motion}$) in more detail. 


### The attraction term

The goal-attraction term measures the distance between the end effector's current position and the goal location. During the animation, the attraction term guides the end-effector towards the goal location. Since its equation is the Euclidean norm (or Euclidean distance), the minimal-solution path, when disregarding obstacles and joint range limits, is a straight line from the end-effector's current location to the goal location. 

$$
\begin{align}
	C_1\left({\bf \Phi}\right) = \|{\bf e}\left({\bf \Phi}\right) - {\bf g}\|
\end{align}
$$


In [2]:
def goal_attraction(e_Phi: np.ndarray, g: np.ndarray) -> float:
    """Computes the value of the goal-attraction cost 
    
    Args:
        e_Phi (np.ndarray): A 3x1 column matrix containing the 
                        (ex,ey,ez) coordinates of the current 
                        location of the end effector.
                        
        g (np.ndarray): A 3x1 column matrix containing the 
                        (gx,gy,gz) coordinates of the 
                        location of the goal point.

    Returns:
        cost (float):   The calculated cost 
    
      """
    
    c = np.linalg.norm(e_Phi - g)
    
        
    return c    # float 

In [3]:
g = np.array([[0],[1],[0]])
e_Phi = np.array([[0],[0],[0]])


goal_attraction(e_Phi, g)

1.0

### The field potential function for obstacle avoidance

The field potential is a *penalty term* that increases its value as the end effector approaches an obstacle. It is defined as follows: 

$$
\begin{align}
        \mathcal{F}_R\left(d\right) = 
	\begin{cases}
		\ln\left(R/d\right), & 0 < d \leq R,\\
		0,                         &d > R.
	\end{cases}
	\label{field_function1}
\end{align}	
$$

In [4]:
def F_R(d: float) -> float:
    """Computes the value of the field potential for 
       obstacle-avoidance cost 
    
    Args:
        d (np.float): Distance between the end-effector and 
                      the obstacle.                        

    Returns:
        cost (float):   The calculated cost 
    
      """
    
    # Assume that all objects have the same size. 
    # Radius (approx) of the object. It might 
    # need to be a bit larger than the actual 
    # radius of the object.
    R = 45 
  
    # TODO 
    if d > R:
        cost = 0
    else:
        cost = np.log(R/d)
        
    return cost   # float 

In [5]:
# End-effector location 
e_Phi = np.array([[3],
                  [0],
                  [0]])

# Location of obstacles 
obstacle = np.array([[10, 10], 
                     [15, 0],
                     [20, 0]])
print(obstacle[:,1])

i = 0
d = np.linalg.norm(e_Phi - obstacle[:,i])
print(d)

print("Cost of obstacle (", i,") = ", F_R(d)) 



[10  0  0]
43.95452195167182
Cost of obstacle ( 0 ) =  0.023506982384087993


### The "range-of-motion" limit function  

The limit function constrains the range of motion of the joints (i.e., angles), and is given by: 

$$
\begin{align}
        \mathcal{L}\left(\phi\right) = 
	\begin{cases}
		\ln\left(\delta/\left(\phi-\phi_\text{min}\right)\right), & \phi_\text{min} < \phi \leq \phi_\text{min} + \delta\\
		0,                         & \phi_\text{min} + \delta < \phi < \phi_\text{max} - \delta\\
		\ln\left(\delta/\left(\phi_\text{max} - \phi\right)\right), & \phi_\text{max} - \delta \leq \phi < \phi_\text{max},\\
	\end{cases}
	\label{limit_function}
\end{align}	
$$

where $\phi$ is the joint angle, $\phi_\text{min}$ and $\phi_\text{max}$ are the limits of that joint, and $\delta$ is the angular distance from each of the limits after which the limit function vanishes. 

In [6]:
def L(phi:float) -> float:
    """Computes the value of the field potential for 
       the joint-limit cost 
    
    Args:
        phi (float): Joint angle 

    Returns:
        cost (float): The calculated cost 
    
      """

    # Assume all limits and delta to be the same
    phi_min = 90        # Joint's minimum angle limit.                        
    
    phi_max = 270       # Joint's maximum angle limit.
    
    delta   = 45       # The angular distance from each of 
                       # the limits after which the limit 
                       # function vanishes.
    
    # TODO
    if phi_min < phi and phi <= (phi_min + delta):
        c = np.log(delta / (phi - phi_min))
    elif (phi_max - delta) < phi and phi < phi_max:
        c = np.log(delta / (phi_max - phi))
    else:
        c = 0
    
    return c   # float 

In [8]:
L(30)

0

In [17]:
def getTransformation(t, theta, rot):

    if theta == -1:  # no rotation
        R = np.eye(3)

    else:
        #c = math.cos(math.radians(theta))
        #s = math.sin(math.radians(theta))
        c = np.cos(theta * np.pi / 180)
        s = np.sin(theta * np.pi / 180)
        
        if rot == 'x':
          R = np.array(
              [[1, 0, 0],
              [0, c, -s],
              [0, s, c]]
          )
        elif rot == 'y':
          R = np.array(
              [[c, 0, s],
              [0, 1, 0],
              [-s, 0, c]]
          )
        else: # rot = 'z'
          R = np.array(
              [[c, -s, 0],
              [s, c, 0],
              [0, 0, 1]]
          )

    T = np.block([[R, t],
                  [np.zeros((1, 3)), 1]])

    return T

In [18]:
# Find (x,y,z) location of ee in global coordinates
def end_effector(Phi):
    l1 = 3
    l2 = 6
    l3 = 5
    l4 = 4
    p1 = np.array([[3],[2],[0]])
    p2 = np.array([[0],[0],[l1]])
    p3 = np.array([[0],[0],[l2]])
    p4 = np.array([[0],[0],[l3]])
    p5 = np.array([[0],[0],[l4]])
    
    # Local transformation matrices 
    
     
    T_01 = getTransformation(p1, Phi[0],'y')
    T_12 = getTransformation(p2, Phi[1],'y')
    T_23 = getTransformation(p3, Phi[2],'y')
    T_34 = getTransformation(p4, Phi[3],'y')

    # Local-to-global transformation matrices
    T_04 = T_01@T_12@T_23@T_34
    
    e = T_04[:3,3]
    print('E:',e)
    
    return e 

In [19]:
# Test array of joint angles 
Phi = np.array([10,20,30,40])   

# End-effector position
end_effector(Phi)

E: [10.85107155  2.         10.65057568]


array([10.85107155,  2.        , 10.65057568])

In [10]:
# Cost function 
#def C(Phi, goal_location): 

# Goal (target) location 
g = np.array([[30],[50],[10]])

# Current joint-angle configuration
Phi = np.array([[0],[11],[22],[33]])

# End-effector location 
e_Phi = end_effector(Phi)

# Obstacles 
num_obstacles = 3

# Location of obstacles 
obstacle = np.array([[10,10,10], 
                     [15,10,10], 
                     [20,20,10]])



cost = 0

cost = np.linalg.norm(e_Phi - g) 
print(cost)


cost += sum(F_R(np.linalg.norm(e_Phi - obstacle[:,j])) for j in range(num_obstacles))
print(cost)

cost += sum(L(phi) for phi in Phi)  
print(cost)
#return cost



59.16079783099616
59.62515445693251
59.62515445693251


### Numerical approximation of the Jacobian matrix 

The first column vector of this Jacobian describes how the end-effector position changes for small changes in joint angle $\phi_1$. Similarly, the second column vector describes how the end-effector position changes for small changes in joint angle $\phi_2$. Note that each component of the Jacobian matrix in Equation $\ref{jacobian_e}$ is equivalent to a 1st-order derivative of a scalar function of a single variable, and can be approximated numerically using finite differences. The approximated Jacobian is then: 

$$
\begin{align}
          J\left({\bf e},{\bf \Phi}\right) \approx  
          \begin{bmatrix}
          	 \dfrac{\Delta{\bf e}}{\Delta\phi_1}  & \dfrac{\Delta{\bf e}}{\Delta\phi_2}  & \dfrac{\Delta{\bf e}}{\Delta\phi_3}  & \dfrac{\Delta{\bf e}}{\Delta\phi_4}
          \end{bmatrix} =           
          \begin{bmatrix}
          	  \dfrac{\Delta e_x}{\Delta\phi_1}  & \dfrac{\Delta e_x}{\Delta\phi_2}   & \dfrac{\Delta e_x}{\Delta\phi_3}  & \dfrac{\Delta e_x}{\Delta\phi_4}\\[1em]
          	 \dfrac{\Delta e_y}{\Delta\phi_1}  & \dfrac{\Delta e_y}{\Delta\phi_2}   & \dfrac{\Delta e_y}{\Delta\phi_3}  & \dfrac{\Delta e_y}{\Delta\phi_4}\\[1em]	 
          	 \dfrac{\Delta e_z}{\Delta\phi_1}  & \dfrac{\Delta e_z}{\Delta\phi_2}   & \dfrac{\Delta e_z}{\Delta\phi_3}  & \dfrac{\Delta e_z}{\Delta\phi_4}	 
	 \end{bmatrix}.          
	\label{jacobian_e_approx}
\end{align}
$$

for a small $\Delta \phi_i, i=1,\dots,4$, where the elements of the matrix in ($\ref{jacobian_e_approx}$) are forward finite differences. We can also write the columns of the Jacobian matrix in terms of ${\bf e}\left(\Phi\right)$ as follows:

$$
\begin{align}
        \frac{\Delta {\bf e}}{\Delta \phi_i} = \frac{{\bf e}\left({\bf \Phi}+\Delta {\bf \Phi}_i\right) - {\bf e}\left({\bf \Phi}\right)}{\Delta \phi_i},
	\label{approxExample}
\end{align}	
$$

for small $\Delta \phi_i$, and $i=1,\dots,4$. Note that the displacement of ${\bf \Phi}$ by $\Delta \phi_i$ in the calculation of ${\bf e}\left({\bf \Phi}+\Delta {\bf \Phi}_i\right)$ in Equation $\ref{approxExample}$ is applied only to the corresponding joint angle $\phi_i$ which is the $i^{th}$ component of ${\bf \Phi}$, and not to all components. For example, the position of the end-effector displaced by $\Delta {\bf \Phi}_1$ is:

$$
\begin{align}
       {\bf e}\left({\bf \Phi}+\Delta {\bf \Phi}_1\right) = {\bf e}\left(
       \begin{bmatrix}
       		\phi_1\\   \phi_2\\ \phi_3\\   \phi_4
       \end{bmatrix}
      +
       \begin{bmatrix}
       		\Delta \phi_1\\   0\\ 0\\   0
       \end{bmatrix}      
      \right).
	\label{incrementPhi}
\end{align}
$$

The displacements for $\phi_2$, $\phi_3$, and $\phi_4$ are calculated analogously. 

In [11]:
def J(end_effector_function, Phi):
    """Computes the value of the function at x
    
    Args:
        end_effector_function: handle to end_effector function 
 
    Returns:
        Jacobian (np.ndarray): A 3x4 Jacobian matrix
    """
    
    # TODO
    
    Jacobian = np.eye(3,4)    
    return Jacobian


### The gradient-descent solution for inverse-kinematics

![ik_algorithm](figs/ik_algorithm.png)

In [12]:
def gradient_descent_IK(): 
    """Solves the inverse-kinematics problem using the gradient descent
    """


    
    return 